In [0]:
dbutils.fs.ls("/mnt/bronze")

Out[2]: [FileInfo(path='dbfs:/mnt/bronze/etl/', name='etl/', size=0, modificationTime=0)]

In [0]:
%fs ls dbfs:/mnt/bronze/etl/car_price/

path,name,size,modificationTime
dbfs:/mnt/bronze/etl/car_price/car_price.parquet,car_price.parquet,197984,1692740467000


In [0]:
file_path = "/mnt/bronze/etl/car_price/car_price.parquet"

In [0]:
#Read the data from ADLS-GEN-2 into a spark dataframe

spark_DF = spark.read.parquet(file_path)

In [0]:
spark_DF.show(3)

+--------------------+-------+---------+---------+------+-------+-----------------+------+-------------+-------+-----------+-----------------+----------------+-----+--------------------+----------+-----+-------------------+-----------+---------------+
|              car_id|  price|fuel_type|gear_type|  make|  model|year_manufactured|colour|    condition|mileage|engine_size|selling_condition|bought_condition|  car|                trim|drivetrain|seats|number_of_cylinders|horse_power|registered_city|
+--------------------+-------+---------+---------+------+-------+-----------------+------+-------------+-------+-----------+-----------------+----------------+-----+--------------------+----------+-----+-------------------+-----------+---------------+
|icwAaDOmX573WiRRn...|2812500|   Petrol|Automatic|Toyota|Corolla|             2004|Silver|Nigerian Used| 251732|       1800|       Registered|      Registered| null|                null|      null| null|               null|       null|         

In [0]:
spark_DF.createOrReplaceTempView('car_price_table')

In [0]:
%sql

SELECT * FROM car_price_table LIMIT 5

car_id,price,fuel_type,gear_type,make,model,year_manufactured,colour,condition,mileage,engine_size,selling_condition,bought_condition,car,trim,drivetrain,seats,number_of_cylinders,horse_power,registered_city
icwAaDOmX573WiRRnLFPT237,2812500,Petrol,Automatic,Toyota,Corolla,2004,Silver,Nigerian Used,251732,1800,Registered,Registered,null,null,null,null,null,null,null
ByZ3NlPy2x0arAIyrKazClUS,5000000,Petrol,Automatic,Acura,MDX,2007,Green,Foreign Used,224589,3700,Imported,Imported,SUV,null,null,null,null,null,null
2zG6Mh3kwyoTvWoeKHkV1oFH,7592000,Petrol,Automatic,Acura,TL,2012,White,Foreign Used,111334,3500,Imported,Imported,Sedan,SH-AWD Automatic Tech Package,All Wheel,5,6,309,null
7APLHOPlI93qsYK0ST4D4wB2,9880000,Petrol,Automatic,Land Rover,Range Rover,2012,Gray,Nigerian Used,72682,4000,Registered,Imported,SUV,null,null,null,null,null,null
zIrIDPJLd46eRQf9JFacc0e,4160000,Petrol,Automatic,Ford,Edge,2013,Gray,Nigerian Used,92118,3500,Registered,Registered,SUV,null,null,null,null,null,Lagos


## Perform Transformation on the Table

In [0]:
display(spark_DF.describe())

summary,car_id,price,fuel_type,gear_type,make,model,year_manufactured,colour,condition,mileage,engine_size,selling_condition,bought_condition,car,trim,drivetrain,seats,number_of_cylinders,horse_power,registered_city
count,3722,3722,3721,3720,3722,3722,3722,3717,3722,3663,3661,3722,3722,1767,879,838,788,826,755,1901
mean,null,4514644.057227297,null,null,null,259.85897435897436,2008.1458893068243,null,null,194984.14387114387,11080.622780661022,null,null,null,178.5752293577982,null,5.285532994923858,5.157384987893463,208.8317880794702,0.0
stddev,null,4297011.880107726,null,null,null,192.0542891563476,4.205669958219907,null,null,139575.5136287561,109112.70744888122,null,null,null,176.4195730910344,null,0.800853322015107,1.2027947010262343,70.94092544714373,0.0
min,10pgkeim9sa8K2pg06xSwLBx,550000,Diesel,Automatic,Acura,116i,1979,Beige,Brand New,1,180,Brand new,Brand new,Bus,1.4,4x2,2,4,83,0
max,zwrhlFXF8Wgt3VJu3aHnrorN,62400000,Petrol,Manual,Volvo,xB,2022,Yellow,Nigerian Used,2456318,3500000,Registered,Registered,Van,xDrive 35i,Rear Wheel,8,8,585,yobe


In [0]:
spark_DF.printSchema()

root
 |-- car_id: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- fuel_type: string (nullable = true)
 |-- gear_type: string (nullable = true)
 |-- make: string (nullable = true)
 |-- model: string (nullable = true)
 |-- year_manufactured: integer (nullable = true)
 |-- colour: string (nullable = true)
 |-- condition: string (nullable = true)
 |-- mileage: integer (nullable = true)
 |-- engine_size: integer (nullable = true)
 |-- selling_condition: string (nullable = true)
 |-- bought_condition: string (nullable = true)
 |-- car: string (nullable = true)
 |-- trim: string (nullable = true)
 |-- drivetrain: string (nullable = true)
 |-- seats: integer (nullable = true)
 |-- number_of_cylinders: integer (nullable = true)
 |-- horse_power: integer (nullable = true)
 |-- registered_city: string (nullable = true)



In [0]:
#Using Pandas to check for columns with null values


pd_df = spark_DF.toPandas()

In [0]:
pd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3722 entries, 0 to 3721
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   car_id               3722 non-null   object 
 1   price                3722 non-null   int32  
 2   fuel_type            3721 non-null   object 
 3   gear_type            3720 non-null   object 
 4   make                 3722 non-null   object 
 5   model                3722 non-null   object 
 6   year_manufactured    3722 non-null   int32  
 7   colour               3717 non-null   object 
 8   condition            3722 non-null   object 
 9   mileage              3663 non-null   float64
 10  engine_size          3661 non-null   float64
 11  selling_condition    3722 non-null   object 
 12  bought_condition     3722 non-null   object 
 13  car                  1767 non-null   object 
 14  trim                 879 non-null    object 
 15  drivetrain           838 non-null    o

In [0]:
#columns to drop
cols = ('car','trim','drivetrain','seats','number_of_cylinders','horse_power')

In [0]:
spark_DF_T = spark_DF.drop(*cols)

In [0]:
pd_df = spark_DF_T.toPandas()

In [0]:
pd_df.columns[pd_df.isna().any()].tolist()

Out[25]: ['fuel_type',
 'gear_type',
 'colour',
 'mileage',
 'engine_size',
 'registered_city']

In [0]:
pd_df.head()

,car_id,price,fuel_type,gear_type,make,model,year_manufactured,colour,condition,mileage,engine_size,selling_condition,bought_condition,registered_city
0,icwAaDOmX573WiRRnLFPT237,2812500,Petrol,Automatic,Toyota,Corolla,2004,Silver,Nigerian Used,251732.0,1800.0,Registered,Registered,None
1,ByZ3NlPy2x0arAIyrKazClUS,5000000,Petrol,Automatic,Acura,MDX,2007,Green,Foreign Used,224589.0,3700.0,Imported,Imported,None
2,2zG6Mh3kwyoTvWoeKHkV1oFH,7592000,Petrol,Automatic,Acura,TL,2012,White,Foreign Used,111334.0,3500.0,Imported,Imported,None
3,7APLHOPlI93qsYK0ST4D4wB2,9880000,Petrol,Automatic,Land Rover,Range Rover,2012,Gray,Nigerian Used,72682.0,4000.0,Registered,Imported,None
4,zIrIDPJLd46eRQf9JFacc0e,4160000,Petrol,Automatic,Ford,Edge,2013,Gray,Nigerian Used,92118.0,3500.0,Registered,Registered,Lagos


In [0]:
#replacing null values in the mileage column with the mean of the mileage column
from pyspark.sql.functions import mean

mean_mileage = round(spark_DF_T.select(mean(spark_DF.mileage)).collect()[0][0],1)
spark_DF_T = spark_DF_T.na.fill(value=mean_mileage,
           subset=["mileage"])


In [0]:
#Fill all the categorical columns with null values

spark_DF_T2 = spark_DF_T.na.fill("Petrol",["fuel_type"]). \
    na.fill("Automatic",["gear_type"]). \
        na.fill("Gray",["colour"]). \
            na.fill("Unknown",["registered_city"])

In [0]:
pd_df = spark_DF_T2.toPandas()
pd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3722 entries, 0 to 3721
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   car_id             3722 non-null   object 
 1   price              3722 non-null   int32  
 2   fuel_type          3722 non-null   object 
 3   gear_type          3722 non-null   object 
 4   make               3722 non-null   object 
 5   model              3722 non-null   object 
 6   year_manufactured  3722 non-null   int32  
 7   colour             3722 non-null   object 
 8   condition          3722 non-null   object 
 9   mileage            3722 non-null   int32  
 10  engine_size        3661 non-null   float64
 11  selling_condition  3722 non-null   object 
 12  bought_condition   3722 non-null   object 
 13  registered_city    3722 non-null   object 
dtypes: float64(1), int32(3), object(10)
memory usage: 363.6+ KB


In [0]:
#Replacing Null values in engine size column with the mode
from pyspark.sql.functions import mode

engine_size_mode = spark_DF_T2.select(mode(spark_DF_T2.engine_size)).collect()[0][0]
spark_cleaned_DF = spark_DF_T2.na.fill(value=engine_size_mode,
           subset=["engine_size"])

In [0]:
#Checking to see if there any null values left in our dataframe
pd_df = spark_cleaned_DF.toPandas()
pd_df.isnull().sum()

Out[43]: car_id               0
price                0
fuel_type            0
gear_type            0
make                 0
model                0
year_manufactured    0
colour               0
condition            0
mileage              0
engine_size          0
selling_condition    0
bought_condition     0
registered_city      0
dtype: int64

In [0]:
spark_cleaned_DF.show(5)

+--------------------+-------+---------+---------+----------+-----------+-----------------+------+-------------+-------+-----------+-----------------+----------------+---------------+
|              car_id|  price|fuel_type|gear_type|      make|      model|year_manufactured|colour|    condition|mileage|engine_size|selling_condition|bought_condition|registered_city|
+--------------------+-------+---------+---------+----------+-----------+-----------------+------+-------------+-------+-----------+-----------------+----------------+---------------+
|icwAaDOmX573WiRRn...|2812500|   Petrol|Automatic|    Toyota|    Corolla|             2004|Silver|Nigerian Used| 251732|       1800|       Registered|      Registered|        Unknown|
|ByZ3NlPy2x0arAIyr...|5000000|   Petrol|Automatic|     Acura|        MDX|             2007| Green| Foreign Used| 224589|       3700|         Imported|        Imported|        Unknown|
|2zG6Mh3kwyoTvWoeK...|7592000|   Petrol|Automatic|     Acura|         TL|       

## Moving the Transformed Data to the Silver level in Delta format

In [0]:
output_path = "/mnt/silver/etl/car_price/"
spark_cleaned_DF.write.format('delta').mode("overwrite").save(output_path)